In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://blog.langchain.dev/langgraph/")

docs =loader.load()

In [3]:
docs

[Document(page_content='\n\n\nLangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024\n\n\n\n\n\nTL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent ru

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

In [6]:
documents

[Document(page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024', metadata={'source': 'https://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}),
 Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primaril

In [7]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(documents, embeddings)

In [8]:
retriever = vector.as_retriever()

In [9]:
retriever.get_relevant_documents(
    "what is LangGraph?"
)[0]

Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\'ve heard requests for, and examples of implementing those. We will finish with a preview of what we will be releasing next.MotivationOne of the big va

In [10]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_search",
    "Search for information about LangGraph. For any question\
 about LangGraph, you must use this tool!",
)

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

search=TavilySearchResults()

In [13]:
search.invoke("what is the weather in istanbul?")

[{'url': 'https://www.whereandwhen.net/when/middle-east/turkey/istanbul/february/',
  'content': 'Best time to go to Istanbul? Weather in Istanbul in february 2024  How was the weather last february? Here is the day by day recorded weather in Istanbul in february 2023:  Seasonal average climate and temperature of Istanbul in february  Weather data for Istanbul for february are derived from an average of the weather forecast since 2009 in Istanbul.Weather in Istanbul in february 2024. The weather in Istanbul in the month of february comes from statistical datas on the last years. You can view the weather statistics the entire month, but also navigating through the tabs for the beginning, the middle and the end of the month. ... 10-02-2023 38°F to 41°F. 11-02-2023 36°F to 41°F. 12-02 ...'},
 {'url': 'https://www.weather25.com/europe/turkey/istanbul',
  'content': "will give you an idea of weather trends in Istanbul. For example, the weather in Istanbul in February 2024.  The weather in I

In [14]:
tools = [retriever_tool, search]

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

In [16]:
from langchain import hub 

prompt = hub.pull("hwchase17/openai-functions-agent")

In [17]:
prompt.messages 

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
from langchain.agents import create_openai_functions_agent

agent=create_openai_functions_agent(model, tools, prompt)

In [19]:
from langchain.agents import AgentExecutor 

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose =True
)

In [20]:
agent_executor.invoke(
    {
        "input":"Hi!"
    }
)



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi!', 'output': 'Hello! How can I assist you today?'}

In [21]:
agent_executor.invoke(
    {
        "input":"what is the weather in london?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in London'}`


[{'url': 'https://weatherspark.com/h/y/45062/2024/Historical-Weather-during-2024-in-London-United-Kingdom', 'content': '2024 Weather History in London United Kingdom  Cloud Cover in 2024 in London Observed Weather in 2024 in London Hours of Daylight and Twilight in 2024 in London  London Temperature History 2024 Hourly Temperature in 2024 in London Compare London to another city:  Hourly Wind Speed in 2024 in London Hourly Wind Direction in 2024 in London Atmospheric Pressure in 2024 in London52 °F cold Precipitation No Report Wind 15.0 mph moderate breeze Wind Dir. 250 deg, WSW Cloud Cover Overcast 1,400 ft Raw: EGLL 030720Z AUTO 25013KT 9999 OVC014 11/08 Q1027 NOSIG This report shows the past weather for London, providing a weather history for 2024.'}, {'url': 'https://www.whereandwhen.net/when/europe/england/london/february/', 'content': 'Best time to go to London?

{'input': 'what is the weather in london?',
 'output': "I found some information about the weather in London. Here are a few sources you can check for more details:\n\n1. [WeatherSpark](https://weatherspark.com/h/y/45062/2024/Historical-Weather-during-2024-in-London-United-Kingdom): This website provides historical weather data for London in 2024, including temperature, wind speed, and cloud cover.\n\n2. [WhereAndWhen.net](https://www.whereandwhen.net/when/europe/england/london/february/): This website offers information about the weather in London in February 2024, including daily average temperature and precipitation.\n\n3. [Candace Abroad](https://candaceabroad.com/what-is-the-weather-like-in-london/): This blog post provides information about the weather in London in 2023, specifically during the winter months of January and February.\n\n4. [Met Office](https://www.metoffice.gov.uk/weather/forecast/gcpvj0v07): The official website of the UK's national weather service provides a sev

 'output': "I found some information about the weather in London. Here are a few sources you can check for more details:\n\n1. [WeatherSpark](https://weatherspark.com/h/y/45062/2024/Historical-Weather-during-2024-in-London-United-Kingdom): This website provides historical weather data for London in 2024, including temperature, wind speed, and cloud cover.\n\n2. [WhereAndWhen.net](https://www.whereandwhen.net/when/europe/england/london/february/): This website offers information about the weather in London in February 2024, including daily average temperature and precipitation.\n\n3. [Candace Abroad](https://candaceabroad.com/what-is-the-weather-like-in-london/): This blog post provides information about the weather in London in 2023, specifically during the winter months of January and February.\n\n4. [Met Office](https://www.metoffice.gov.uk/weather/forecast/gcpvj0v07): The official website of the UK's national weather service provides a seven-day weather forecast for London, including temperature, rain, wind, visibility, humidity, and UV index.\n\nPlease note that the information provided may vary depending on the source and the specific time period you are interested in."}

In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!"),
]

In [23]:
agent_executor.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how",
    }
)



> Entering new AgentExecutor chain...

Invoking: `langgraph_search` with `{'query': 'LangGraph for agent runtimes'}`


TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we've implemented already. We will then highlight a few of the common modifications to these runtimes we've heard requests for, and examples of implementing th

{'chat_history': [HumanMessage(content='Can I use LangGraph for agent runtimes?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangGraph is a module built on top of LangChain that enables the creation of cyclical graphs, which are often needed for agent runtimes. LangGraph allows you to create state machines by specifying them as graphs.\n\nHere is a brief overview of how to use LangGraph for agent runtimes:\n\n1. Initialize a StateGraph object by passing in a state definition. This state definition represents a central state object that is updated over time.\n\n2. Add nodes to the graph using the `graph.add_node(name, value)` syntax. The name parameter is a string used to refer to the node, and the value parameter is a function or LCEL runnable that will be called. The function/LCEL should accept a dictionary in the same form as the State object as input and output a dictionary with keys of the State object to update.\n\n3. Add edges to connect the nodes and cre

 'output': 'LangGraph is a module built on top of LangChain that enables the creation of cyclical graphs, which are often needed for agent runtimes. LangGraph allows you to create state machines by specifying them as graphs.\n\nHere is a brief overview of how to use LangGraph for agent runtimes:\n\n1. Initialize a StateGraph object by passing in a state definition. This state definition represents a central state object that is updated over time.\n\n2. Add nodes to the graph using the `graph.add_node(name, value)` syntax. The name parameter is a string used to refer to the node, and the value parameter is a function or LCEL runnable that will be called. The function/LCEL should accept a dictionary in the same form as the State object as input and output a dictionary with keys of the State object to update.\n\n3. Add edges to connect the nodes and create the graph. There are three types of edges:\n   - Starting Edge: Connects the start of the graph to a particular node.\n   - Normal Edges: Connects one node to another, indicating that the second node should always be called after the first node.\n   - Conditional Edges: Uses a function to determine which node to go to next. The function should return a string, and a mapping is used to map the output of the function to another node.\n\n4. Compile the graph into a runnable using the `graph.compile()` method. This returns a runnable that can be called in the same manner as a LangChain chain.\n\nAdditionally, LangGraph provides an Agent Executor that allows you to use existing LangChain agents and modify the internals of the AgentExecutor.\n\nFor more detailed information and examples, you can refer to the LangGraph documentation and the LangChain blog post on LangGraph.'}